In [ ]:
import json
import psycopg2
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
# plt.style.use("clint.mpl")
import matplotlib.dates as md

# silence annoying warning about plotting pd.datetime objects with mpl
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

In [ ]:
with open("config.json") as f:
    config = json.load(f)

db_conn = psycopg2.connect(dbname=config["db_name"], 
                           user=config["db_user"], 
                           password=config["password"], 
                           host=config["cage_daq"])
db_cursor = db_conn.cursor()

In [ ]:
cmd = "SELECT * FROM endpoint_id_map;"
db_cursor.execute(cmd)
record = db_cursor.fetchall()
df_end = pd.DataFrame(record, columns=["row", "endpoint_name", "data_type"])
# df_end

In [ ]:
endpoints = ['cage_pressure', 'cage_coldPlate_temp', 'cage_heater_temp', 'cage_topHat_temp', 'cage_motor_temp']

# t_earlier = '2021-09-03T012:00'
# t_later = '2021-09-08T00:00'

t_earlier = '2021-11-20T012:00'
t_later = '2021-11-23T00:00'

dfs = {}

for pt in endpoints:
    query = f"SELECT value_cal, timestamp FROM numeric_data "
    query += f"WHERE endpoint_name='{pt}'"
    query += f"AND timestamp>='{t_earlier}' and timestamp<='{t_later}';"

    db_cursor.execute(query)
    record = db_cursor.fetchall()
    df = pd.DataFrame(record, columns=[pt, 'timestamp']) 
    dfs[pt] = df
    
print(f'Found {len(dfs[endpoints[0]])} entries.')

In [ ]:
%matplotlib widget
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FuncFormatter

# ts = dfs['cage_pressure']['timestamp'].dt.tz_localize('US/Pacific')

f, p = plt.subplots(2, 1, figsize=(8,8), gridspec_kw={'height_ratios': [1, 1.5]}, tight_layout=True)

# f.suptitle('CAGE Pump & Bake, Sep 3--6, 2021')
f.suptitle('CAGE Pump & Bake, Nov 19--22, 2021')

df = dfs['cage_pressure']
p[0].plot(df.timestamp.dt.tz_localize('US/Pacific'), df.cage_pressure)
p[0].grid()
p[0].set_xlabel('Date (US/Pacific)', ha='right', x=1)
p[0].set_ylabel('Pressure (mbar)')
# p[0].Axes.ticklabel_format(axis='y', style='sci')
p[0].ticklabel_format(style='sci', axis='y')

# date_form = DateFormatter("%m-%d %H:%m")
# p[0].xaxis.set_major_formatter(date_form)

def scientific(x, pos):
    # x:  tick value - ie. what you currently see in yticks
    # pos: a position - ie. the index of the tick (from 0 to 9 in this example)
    return '%.1e' % x
scientific_formatter = FuncFormatter(scientific)
p[0].yaxis.set_major_formatter(scientific_formatter)

for ept in endpoints[1:]:
    df = dfs[ept]
    p[1].plot(df.timestamp.dt.tz_localize('US/Pacific'), df[ept], label=ept)
    
p[1].legend()
p[1].grid()
p[1].set_ylabel('Temp (C)')
p[1].set_xlabel('Date (US/Pacific)', ha='right', x=1)